In [1]:
%%writefile workspace.json
{
"subscription_id" : "fe38c376-b42a-4741-9e7c-f5d7c31e5873",
"resource_group" : "yelizkilinc-rg",
"workspace_name" : "aml-prod"
}

Overwriting workspace.json


In [2]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azureml.core import Workspace

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

try:
    ml_client = MLClient.from_config(credential=credential, path="workspace.json")
except Exception as ex:
    raise Exception(
        "Failed to create MLClient from config file. Please modify and then run the above cell with your AzureML Workspace details."
    ) from ex
    # ml_client = MLClient(
    #     credential=credential,
    #     subscription_id="",
    #     resource_group_name="",
    #     workspace_name=""
    # )

ws = Workspace(
    subscription_id=ml_client.subscription_id,
    resource_group=ml_client.resource_group_name,
    workspace_name=ml_client.workspace_name,
)
print(ml_client)

Found the config file in: workspace.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f53e7feb3a0>,
         subscription_id=fe38c376-b42a-4741-9e7c-f5d7c31e5873,
         resource_group_name=yelizkilinc-rg,
         workspace_name=aml-prod)


In [ ]:
import uuid

AOAI_endpoint = "https://openaiykus.openai.azure.com/" 
key_value="9ed9191bbdae445cbe69af2709668995"

keyvault = ws.get_default_keyvault()
keyvault.set_secret(name="aoai-endpoint", value = AOAI_endpoint)
keyvault.set_secret(name="key",value=key_value)


In [21]:
aoai_endpoint=keyvault.get_secret(name="aoai-endpoint")
aoai_key=keyvault.get_secret(name="key")

In [14]:
import openai

In [22]:
# Set up Azure OpenAI
openai.api_type = "azure"
openai.api_base = aoai_endpoint # Api base is the 'Endpoint' which can be found in Azure Portal where Azure OpenAI is created. It looks like https://xxxxxx.openai.azure.com/
openai.api_version = "2022-12-01"
openai.api_key = aoai_key

In [39]:
prompt_prefix = """ 
  Extract keywords from only this text: 
"""

prompt = prompt_prefix + "Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.  The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn."
prompt

' \n  Extract keywords from only this text: \nQuarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.  The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn.'

In [42]:
response = openai.Completion.create(
  deployment_id="gpt-35-turbo", # has to be deployment_id
  prompt= prompt,
  temperature=1,
  max_tokens=10,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0
)


In [43]:
# print response
response['choices'][0]['text']

'  Its profits were buoyed by one-off gains'